# 🤖 Sidekick: Your AI Assistant with Built-in Quality Control

## 🎯 Project Overview

**Sidekick** is an intelligent AI assistant that doesn't just answer questions—it **validates its own work** and **improves iteratively** until it meets your success criteria. Think of it as an AI with a built-in quality assurance system that ensures every response is accurate, complete, and truly helpful.

## 🚀 What Makes This Special?

Unlike traditional chatbots that give you a single answer and hope for the best, Sidekick uses a **Worker-Evaluator feedback loop**:

1. **🔨 Worker Agent**: Receives your task, uses tools (web search, notifications), and generates an answer
2. **🔍 Evaluator Agent**: Critically reviews the worker's output against your success criteria
3. **🔄 Self-Improvement Loop**: If the evaluator finds issues, the worker gets detailed feedback and tries again
4. **✅ Quality Guarantee**: The loop continues until the work meets your standards OR it needs clarification from you

## 🛠️ Capabilities

- **🌐 Web Search**: Finds real-time information using Google Search API
- **📱 Push Notifications**: Sends alerts to your phone via Pushover
- **💾 Conversation Memory**: Remembers context across interactions
- **🎯 Custom Success Criteria**: Define what "good" means for each task
- **🔄 Automatic Retries**: Self-corrects based on evaluator feedback

## 📚 Learning Objective

This notebook rebuilds the Sidekick project using **Microsoft Agent Framework** (originally built with LangGraph). You'll learn:

- How to orchestrate multi-agent workflows with explicit control flow
- Implementing feedback loops for self-improving AI systems
- Building production-ready agents with tool integration
- Creating quality assurance mechanisms in AI workflows
- Working with async/await patterns in agent frameworks

## 🎓 The Architecture

```
User Request → Worker Agent → Tools (Search/Notify) → Evaluator Agent
                                                              ↓
                                                         Meets Criteria?
                                                              ↓
                                                    ┌─────────┴─────────┐
                                                    ↓                   ↓
                                              ✅ Success         ❌ Retry with Feedback
```

Let's dive in! 🚀


---
📢 Discover more Agentic AI notebooks on my [GitHub repository](https://github.com/lisekarimi/agentverse) and explore additional AI projects on my [portfolio](https://lisekarimi.com).

## === IMPORTS AND SETUP ===

In [ ]:
# uv add agent-framework --pre

In [ ]:
import asyncio
import gradio as gr
from datetime import datetime
import os
from dotenv import load_dotenv
from agent_framework.openai import OpenAIChatClient
from typing import Annotated
import requests
from agent_framework import (
    ChatAgent,
    ChatMessageStore,
    WorkflowBuilder,
    Executor,
    WorkflowContext,
    handler,
    Case,
    Default
)
from pydantic import BaseModel, Field
from typing_extensions import Never

In [ ]:
load_dotenv(override=True)
MODEL_ID = "gpt-4o-mini"
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
serper_api_key = os.getenv("SERPER_API_KEY")

# Validate required environment variables
if not serper_api_key:
    print("Warning: SERPER_API_KEY is not configured. Please set it in your .env file.")
if not pushover_user or not pushover_token:
    print("Warning: PUSHOVER_USER and PUSHOVER_TOKEN are required. Please set them in your .env file.")

## === TOOLS ===

In [ ]:
# Define the Pushover tool
def send_pushover_notification(
    message: Annotated[str, Field(description="The message to send")],
    title: Annotated[str, Field(description="The notification title")] = "Agent Alert"
) -> str:
    """Send a push notification via Pushover."""



    response = requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": pushover_token,
            "user": pushover_user,
            "message": message,
            "title": title
        }
    )

    return "✅ Notification sent" if response.status_code == 200 else "❌ Failed"

In [ ]:
# Define the serper search tool
def google_search(
    query: Annotated[str, Field(description="Search query")]
) -> str:
    """Search Google using Serper API."""
    response = requests.post(
        "https://google.serper.dev/search",
        headers={"X-API-KEY": serper_api_key, "Content-Type": "application/json"},
        json={"q": query}
    )

    if response.status_code == 200:
        data = response.json()
        results = []
        for item in data.get("organic", [])[:3]:
            results.append(f"• {item['title']}: {item['snippet']}")
        return "\n".join(results) if results else "No results found"
    return "Search failed"

## === DATA MODELS ===

In [ ]:
class UserRequest(BaseModel):
    question: str
    success_criteria: str

class WorkerOutput(BaseModel):
    answer: str
    iteration: int = 0

class EvaluatorFeedback(BaseModel):
    success_criteria_met: bool
    user_input_needed: bool
    feedback: str
    worker_answer: str
    iteration: int

## === EXECUTORS ===

In [ ]:
message_store = ChatMessageStore()

# Worker: Does the actual work
class WorkerExecutor(Executor):
    def __init__(self):
        super().__init__(id="worker")
        self.agent = ChatAgent(
            chat_client=OpenAIChatClient(model_id="gpt-4o-mini"),
            instructions=f"""You are a helpful AI assistant with access to tools.

Current date/time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

Your job:
1. If the user's request is unclear or missing details, ask for clarification
2. If clear, use google_search to find information and answer thoroughly
3. If user asks to send notification, use send_pushover_notification

Be detailed and use search results to provide accurate answers.""",
            tools=[google_search, send_pushover_notification],
            chat_message_store_factory=lambda: message_store
        )
        self.iteration_count = 0

    @handler
    async def work(self, data: UserRequest | EvaluatorFeedback, ctx: WorkflowContext[WorkerOutput]) -> None:
        self.iteration_count += 1

        # Build prompt based on input
        if isinstance(data, UserRequest):
            prompt = f"""Task: {data.question}

Success criteria: {data.success_criteria}

Please complete this task."""
        else:  # EvaluatorFeedback
            prompt = f"""Your previous answer didn't meet the success criteria.

Evaluator feedback: {data.feedback}

Please improve your answer."""

        print(f"\n🔨 WORKER (Iteration {self.iteration_count}): Working on task...")
        result = await self.agent.run(prompt, thread_id="sidekick_memory")
        print(f"✅ WORKER: Generated answer ({len(result.text)} chars)")

        output = WorkerOutput(answer=result.text, iteration=self.iteration_count)
        await ctx.send_message(output)

# Evaluator: Checks quality
class EvaluatorExecutor(Executor):
    def __init__(self):
        super().__init__(id="evaluator")
        self.agent = ChatAgent(
            chat_client=OpenAIChatClient(model_id="gpt-4o-mini"),
            instructions="""You are an evaluator that judges if work meets success criteria.

Evaluate the worker's answer and respond with:
SUCCESS: yes/no
USER_INPUT_NEEDED: yes/no (if question is unclear and needs user clarification)
FEEDBACK: [detailed feedback on what needs improvement]

Criteria for SUCCESS:
- Directly answers the user's question
- Meets the success criteria
- Is accurate and complete
- Used search results properly (if needed)

Criteria for USER_INPUT_NEEDED:
- Question is too vague or unclear
- Missing critical information to proceed

If SUCCESS=yes OR USER_INPUT_NEEDED=yes, we stop.
If both are no, worker must retry with your feedback.""",
            chat_message_store_factory=lambda: message_store
        )

    @handler
    async def evaluate(
        self,
        worker_output: WorkerOutput,
        ctx: WorkflowContext[EvaluatorFeedback | str]
    ) -> None:
        # Get the original request from context (would be passed through in real impl)
        # For now, we evaluate based on the answer quality

        print("\n🔍 EVALUATOR: Reviewing answer...")
        result = await self.agent.run(
            f"Evaluate this answer:\n\n{worker_output.answer}",
            thread_id="sidekick_memory"
        )

        # Parse evaluator response
        text = result.text
        success = "SUCCESS: yes" in text or "success: yes" in text.lower()
        needs_input = "USER_INPUT_NEEDED: yes" in text or "user_input_needed: yes" in text.lower()

        # Extract feedback
        feedback_lines = [line for line in text.split("\n") if "FEEDBACK:" in line]
        feedback = feedback_lines[0].split("FEEDBACK:")[-1].strip() if feedback_lines else text

        print("📊 EVALUATOR RESULTS:")
        print(f"   Success: {success}")
        print(f"   Needs user input: {needs_input}")
        print(f"   Feedback: {feedback[:100]}...")

        if success or needs_input:
            # Task complete - send final answer
            final_msg = worker_output.answer
            if needs_input:
                final_msg = f"❓ CLARIFICATION NEEDED:\n\n{worker_output.answer}"
            else:
                final_msg = f"✅ TASK COMPLETE:\n\n{worker_output.answer}"

            await ctx.send_message(final_msg)
        else:
            # Send back to worker with feedback
            print("↩️  EVALUATOR: Sending back to worker for improvement...")
            feedback_obj = EvaluatorFeedback(
                success_criteria_met=False,
                user_input_needed=False,
                feedback=feedback,
                worker_answer=worker_output.answer,
                iteration=worker_output.iteration
            )
            await ctx.send_message(feedback_obj)

# Final Output
class FinalOutputExecutor(Executor):
    @handler
    async def output(self, message: str, ctx: WorkflowContext[Never, str]) -> None:
        await ctx.yield_output(message)


## == BUILD WORKFLOW ===

In [ ]:
worker = WorkerExecutor()
evaluator = EvaluatorExecutor()
final_output = FinalOutputExecutor(id="final_output")

workflow = (
    WorkflowBuilder()
    .set_start_executor(worker)
    .add_edge(worker, evaluator)
    .add_switch_case_edge_group(
        evaluator,
        [
            Case(condition=lambda x: isinstance(x, str), target=final_output),
            Default(target=worker)
        ]
    )
    .build()
)

## === GRADIO ===

In [ ]:
async def run_task(question, success_criteria):
    if not success_criteria:
        success_criteria = "Provide a clear, accurate, and complete answer"

    print("\n" + "="*60)
    print(f"NEW TASK: {question}")
    print(f"SUCCESS CRITERIA: {success_criteria}")
    print("="*60)

    request = UserRequest(question=question, success_criteria=success_criteria)

    result = ""
    async for event in workflow.run_stream(request):
        if event.__class__.__name__ == 'WorkflowOutputEvent':
            result = event.data

    return result

def run(question, success_criteria):
    return asyncio.run(run_task(question, success_criteria))

demo = gr.Interface(
    fn=run,
    inputs=[
        gr.Textbox(label="Your Question/Task", lines=3, placeholder="What do you need?"),
        gr.Textbox(
            label="Success Criteria (optional)",
            lines=2,
            placeholder="What makes a good answer? (leave blank for default)",
            value=""
        )
    ],
    outputs=gr.Textbox(label="Result", lines=15),
    title="🤖 AI Sidekick (Worker → Evaluator Loop)",
    description="Worker does the task → Evaluator checks quality → Loops until approved. Has memory, Google search, and Pushover!",
    examples=[
        ["What are the top 3 AI trends in 2024?", "List 3 specific trends with brief explanations"],
        ["Send me 3 jobs that I can do", "Provide exactly 3 specific job listings that match the user's skills and preferences"],
        ["Find best restaurants in Paris and notify me", "Provide specific restaurant names with ratings"]
    ]
)


In [ ]:
demo.launch()